In [1]:
import sys

import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("../RD-systems-and-test-benches/utils")
import model_fsolve as modf
import model_fsolve_v2 as modf2

import scipy.optimize as sco
import scipy.interpolate as sci

import hx_hydraulic as hxhy
import from_excel as fe
import data_plots as dplt

import fluids as fds
from CoolProp.CoolProp import PropsSI

import openpyxl as opxl
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns
import os
import plotly.graph_objects as go

import copy


In [2]:
import repo_config as rconfig

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import FuncFormatter
import matplotlib.gridspec as gridspec
import matplotlib_inline

import scienceplots

matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
plt.style.use(['science'])

def to_percent(y, _):
    return f"{y * 100:.0f}\%"

# colors = ['olive', 'green', 'blue', 'orange', 'red', 'grey','yellow','bordeaux']
colors = ['olive','orange','blue','red','green','grey']
colors70 = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5)), (0, (3, 5, 1, 5)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd', 'X', 'v', '^', 'P', 'H']

In [ ]:
path = r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Inputs"
file_name = "\V4.5.xlsx"
hw, par, cond = fe.initialize(path, file_name)

tabl, res, df_PL, residuals = modf.PL_fsolve(par, cond) 
print('PL = ', res, ' Pa')

### Parametric M = A_R

In [4]:
M_range = [0.5,0.75,1,1.5,2,2.5,3, 3.5, 4.]

M_res_list = []

for i, M in enumerate(M_range):
    hw.change_man_diameter(math.sqrt( (1/M) * hw.N * (hw.riser.D/2)**2 ) * 2)
    hw.compute_metrics()
    par = hw.make_dict()
    # print(hw.M)

    tabl, res, PL, residuals = modf.PL_fsolve(par, cond) 

    M_res_list.append({'M' : M,
                       'D_man' : hw.man.D,
                       'par' : copy.deepcopy(par),
                       'tabl' : copy.deepcopy(tabl),
                       'PL' : res, 
                       'PL_tabl' : copy.deepcopy(PL),
                       'residuals' : copy.deepcopy(residuals)})
    
for i, M_res in enumerate(M_res_list):
    M_res['tabl']['qx_norm'] = M_res['tabl']['qx'] / (M_res['tabl']['qx'].sum() / hw.N)

In [5]:
# # Plot flow distribution

# fig, ax1 = plt.subplots(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))

# y_name = 'qx_norm'

# color_intensity = [70,100,150]

# for i, M in enumerate( [0.5,0.75,1,1.5,2.5,3.5] ):
#     M_res = M_res_list[i]
#     tabl = M_res['tabl']

#     ax1.plot(tabl.index, tabl[y_name], label = f"M = {M}, D_man = {round(1000*M_res['D_man'],1)} mm",
#              color = colors[i],
#              linestyle = linestyles[i],
#              linewidth = 2.
#             )

# # # Create the first row of the legend (for the lines)
# # first_legend = fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=3, frameon=False,
# #                           labels=['Model w/ MPG 20\%', 'Model w/ MPG 30\%', 'Model w/ MPG 40\%'],
# #                           handles=[ax1.lines[0], ax1.lines[1], ax1.lines[2]])

# # # Create the second row of the legend (for the scatter plot)
# # second_legend = fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08), ncol=1, frameon=False,
# #                            labels=['Measure'], handles=[ax1.collections[1]])

# # # Add the first legend back to the axes (to avoid being overwritten)
# # ax1.add_artist(first_legend)

# ax1.set_xlabel('Tube number')
# ax1.set_ylabel('Normalized flow rate [-]')

# # ax1.set_ylim(0.92,1.12)

# ax1.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

# fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=2, frameon=False,)



In [6]:
# # Plot PL

# fig, ax1 = plt.subplots(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))

# test_range = M_range
# res_list = M_res_list

# ax1.scatter(test_range,[res['PL'] for res in res_list], color = colors[-1])

# ax1.set_xlabel('M [-]')
# ax1.set_ylabel('PL [Pa]')

# ax1.grid()

In [ ]:
# Define GridSpec layout for the two plots (4 columns: 3 for the first plot, 1 for the second)
fig = plt.figure(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))
gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])  # 3/4 width for the first plot, 1/4 for the second

# First plot: 3/4 of the width
ax1 = fig.add_subplot(gs[0, :-1])  # Use the first 3 columns for the first plot

y_name = 'qx_norm'

for i, M in enumerate([0.5, 0.75, 1, 1.5, 2.5, 3.5]):
    M_res = M_res_list[i]
    tabl = M_res['tabl']
    ax1.plot(tabl.index, tabl[y_name], label=f"$M = {M} \ (D_{{man}} = {round(1000*M_res['D_man'],1)} \: mm)$",
             color=colors[i],
             linestyle=linestyles[i],
             linewidth=2.)

ax1.set_xlabel('Tube number [-]')
ax1.set_ylabel('Normalized flow rate [-]')
ax1.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

# Adding the legend below the first plot
fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=2, frameon=False)

# Second plot: 1/4 of the width
ax2 = fig.add_subplot(gs[0, -1])  # Use the last column for the second plot

test_range = M_range
res_list = M_res_list

ax2.scatter(test_range, [res['PL'] for res in res_list], color=colors70[-1])

ax2.set_xlabel('M [-]')
ax2.set_ylabel('PL [Pa]')
ax2.grid()

# Adjust the layout to ensure alignment of the plots
plt.subplots_adjust(wspace=0.1)  # Adjust horizontal space between plots

# Ensure that both y-axes are aligned
# ax1.get_shared_y_axes().join(ax1, ax2)

# Adjust the layout to ensure better spacing between plots
plt.subplots_adjust(wspace=0.3)  # Increase horizontal space between plots

plt.show()


In [126]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_PL\PL-2.4-parametric-M.png', dpi=600)

### Parametric R_D by changing riser diameter

remove distance inter MPE

choose M = 2

In [130]:
path = r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Inputs"
file_name = "\V4.5_wo_interMPE.xlsx"
hw, par, cond = fe.initialize(path, file_name)

In [134]:
RD_range = [0.05,0.1,0.25,0.5,0.7,1.]

RD_res_list = []

M = 2

for i, RD in enumerate(RD_range):

    hw_local = copy.deepcopy(hw)

    hw_local.change_riser_diameter(hw_local.man.D * RD)

    hw_local.change_N_total(int ( M * (hw_local.man.A / (math.pi*(hw_local.riser.D/2)**2)) ))

    hw_local.compute_metrics()
    # print('M = ', hw_local.M)
    # print('R_D =', hw_local.R_D)

    # print('D_man =', hw_local.man.D)
    # print('D_riser = ', hw_local.riser.D)
    # print('N = ', hw_local.N)

    # print(' ---- ')

    par = hw_local.make_dict()

    tabl, res, df_PL, residuals = modf.PL_fsolve(par, cond) 

    tabl['qx_norm'] = tabl['qx'] / (tabl['qx'].sum() / hw_local.N)

    RD_res_list.append({'M' : M,
                       'D_man' : hw_local.man.D,
                       'par' : copy.deepcopy(par),
                       'tabl' : copy.deepcopy(tabl),
                       'PL' : res, 
                       'df_PL' : copy.deepcopy(PL),
                       'residuals' : copy.deepcopy(residuals)})

In [ ]:
# Main plot: plotting L_f0_range on x-axis
fig, ax1 = plt.subplots(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))

y_name = 'qx_norm'

color_intensity = [70,100,150]

res_list = RD_res_list

for i, k in enumerate(RD_range):
    res = res_list[i]
    tabl = res['tabl']

    tabl['x'] = ((0.5+tabl.index)/(tabl.index[-1]+1)) * hw_local.man.L

    ax1.scatter(round(tabl['x']*1000,1), tabl[y_name], label = f"$R_D = {RD_range[i]} (N = {res['par']['N_riser_per_panel']}, D_{{riser}} = {round(1000*res['par']['D_riser'],1)} \: mm)$",
             color = colors[i],
             marker = markers[i]
            )

ax1.set_xlabel('Position along the manifold [mm]')
ax1.set_ylabel('Normalized flow rate [-]')

ax1.set_xlim(0,957.95)

ax1.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=2, frameon=False,)

In [ ]:
# Define GridSpec layout for the two plots (2 columns: 3/4 for the first plot, 1/4 for the second)
fig = plt.figure(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))
gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])  # 3/4 width for the first plot, 1/4 for the second

# First plot: 3/4 of the width (the new one)
ax1 = fig.add_subplot(gs[0, 0])  # Use the first column (taking 3/4 of space)

y_name = 'qx_norm'
res_list = RD_res_list

for i, k in enumerate(RD_range):
    res = res_list[i]
    tabl = res['tabl']

    tabl['x'] = ((0.5+tabl.index)/(tabl.index[-1]+1)) * hw_local.man.L

    ax1.scatter(round(tabl['x']*1000,1), tabl[y_name], label=f"$R_D = {RD_range[i]} (N = {res['par']['N_riser_per_panel']}, D_{{riser}} = {round(1000*res['par']['D_riser'],1)} \: mm)$",
                color=colors[i],
                marker=markers[i])

ax1.set_xlabel('Position along the manifold [mm]')
ax1.set_ylabel('Normalized flow rate [-]')

ax1.set_xlim(0,957.95)

ax1.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

# Adding the legend below the first plot
fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=2, frameon=False)

# Second plot: 1/4 of the width (scatter plot from the previous example)
ax2 = fig.add_subplot(gs[0, 1])  # Use the second column (taking 1/4 of space)

test_range = RD_range  # Dummy range for this example
res_list = RD_res_list

ax2.scatter(test_range, [res['PL'] for res in res_list], color=colors70[-1])

ax2.set_xlabel('$R_D$ [-]')
ax2.set_ylabel('PL [Pa]')
ax2.grid()

# Adjust the layout to ensure better spacing between plots
plt.subplots_adjust(wspace=0.4)  # Increase horizontal space between plots

plt.show()

In [151]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_PL\PL-2.4-parametric-RD-riser-FDPL.png', dpi=600)

### Parametric R_D by changing manifold diameter

In [ ]:
RD_range = [0.05,0.1,0.25,0.5,0.7,1.]

RDman_res_list = []

M = 2

for i, RD in enumerate(RD_range):

    hw_local = copy.deepcopy(hw)

    hw_local.change_man_diameter(hw_local.riser.D / RD)

    hw_local.change_N_total(int ( M * (hw_local.man.A / (math.pi*(hw_local.riser.D/2)**2)) ))

    hw_local.compute_metrics()
    # print('M = ', hw_local.M)
    # print('R_D =', hw_local.R_D)

    # print('D_man =', hw_local.man.D)
    # print('D_riser = ', hw_local.riser.D)
    # print('N = ', hw_local.N)

    # print(' ---- ')

    par = hw_local.make_dict()

    tabl, res, PL, residuals = modf.PL_fsolve(par, cond) 

    tabl['qx_norm'] = tabl['qx'] / (tabl['qx'].sum() / hw_local.N)

    RDman_res_list.append({'M' : M,
                       'D_man' : hw_local.man.D,
                       'par' : copy.deepcopy(par),
                       'tabl' : copy.deepcopy(tabl),
                       'PL' : res, 
                       'PL_tabl' : copy.deepcopy(PL),
                       'residuals' : copy.deepcopy(residuals)})

In [ ]:
# Main plot: plotting L_f0_range on x-axis
fig, ax1 = plt.subplots(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))

y_name = 'qx_norm'

res_list = RDman_res_list

for i, k in enumerate(RD_range):
    res = res_list[i]
    tabl = res['tabl']

    print(res['par']['D_man'])

    tabl['x'] = ((0.5+tabl.index)/(tabl.index[-1]+1)) * hw_local.man.L

    ax1.scatter(round(tabl['x']*1000,1), tabl[y_name], label = f"$R_D = {RD_range[i]}, N = {res['par']['N_riser_per_panel']}, D_{{riser}} = {round(1000*res['par']['D_riser'],1)} \: mm$",
             color = colors[i],
             marker = markers[i]
            )

ax1.set_xlabel('Position along the manifold [mm]')
ax1.set_ylabel('Normalized flow rate [-]')

ax1.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=2, frameon=False,)

In [ ]:
# Define GridSpec layout for the two plots (2 columns: 3/4 for the first plot, 1/4 for the second)
fig = plt.figure(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))
gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])  # 3/4 width for the first plot, 1/4 for the second

# First plot: 3/4 of the width (the new one)
ax1 = fig.add_subplot(gs[0, 0])  # Use the first column (taking 3/4 of space)

y_name = 'qx_norm'
res_list = RDman_res_list

for i, k in enumerate(RD_range):
    res = res_list[i]
    tabl = res['tabl']
    
    print(res['par']['D_man'])  # Print statement as in the provided code
    
    tabl['x'] = ((0.5 + tabl.index) / (tabl.index[-1] + 1)) * hw_local.man.L

    ax1.scatter(round(tabl['x'] * 1000, 1), tabl[y_name], label=f"$R_D = {RD_range[i]}, N = {res['par']['N_riser_per_panel']}, D_{{riser}} = {round(1000*res['par']['D_riser'], 1)} \: mm$",
                color=colors[i], marker=markers[i])

ax1.set_xlim(0,957.95)
ax1.set_xlabel('Position along the manifold [mm]')
ax1.set_ylabel('Normalized flow rate [-]')
ax1.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

# Adding the legend below the first plot
fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=2, frameon=False)

# Second plot: 1/4 of the width (scatter plot from the previous example)
ax2 = fig.add_subplot(gs[0, 1])  # Use the second column (taking 1/4 of space)

test_range = RD_range

ax2.scatter(test_range, [res['PL'] for res in res_list], color='grey')

ax2.set_xlabel('$R_D$ [-]')
ax2.set_ylabel('PL [Pa]')
ax2.grid()

# Adjust the layout to ensure better spacing between plots
plt.subplots_adjust(wspace=0.3)  # Increase horizontal space between plots

plt.show()

In [168]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_PL\PL-2.4-parametric-RD-man.png', dpi=600)

### Parametric length riser

In [186]:
path = r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Inputs"
file_name = "\V4.5.xlsx"
hw, par, cond = fe.initialize(path, file_name)

In [205]:
L_range = [0.25,0.5,0.75,1,1.25,1.5]

L_res_list = []

for i, test_value in enumerate(test_range):

    hw_local = copy.deepcopy(hw)

    hw_local.riser.L = test_value

    hw_local.compute_metrics()
    par = hw_local.make_dict()

    tabl, res, df_PL, residuals = modf.PL_fsolve(par, cond) 

    tabl['qx_norm'] = tabl['qx'] / (tabl['qx'].sum() / hw_local.N)

    L_res_list.append({'L' : test_value,
                       'par' : copy.deepcopy(par),
                       'tabl' : copy.deepcopy(tabl),
                       'PL' : res, 
                       'df_PL' : copy.deepcopy(df_PL),
                       'residuals' : copy.deepcopy(residuals),
                       }
                       )
    

In [ ]:
# Main plot: plotting L_f0_range on x-axis
fig, ax1 = plt.subplots(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))

y_name = 'qx_norm'

color_intensity = [70,100,150]

res_list = L_res_list

for i, L in enumerate([0.25,0.5,0.75,1,1.5,2.]):
    res = res_list[i]
    tabl = res['tabl']

    tabl['x'] = ((0.5+tabl.index)/(tabl.index[-1]+1)) * hw_local.man.L

    ax1.plot(round(tabl['x']*1000,1), tabl[y_name], label = f"$L_{{riser}} = {round(res['par']['L_riser']*1000,1)} \: mm$",
             color = colors[i],
             linestyle = linestyles[i],
             linewidth = 2.
            )

ax1.set_xlabel('Position along the manifold [mm]')
ax1.set_ylabel('Normalized flow rate [-]')

ax1.set_xlim(0,714.75)

ax1.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=2, frameon=False,)



In [ ]:
# Define GridSpec layout for the two plots (2 columns: 3/4 for the first plot, 1/4 for the second)
fig = plt.figure(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))
gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])  # 3/4 width for the first plot, 1/4 for the second

# First plot: 3/4 of the width (the new one)
ax1 = fig.add_subplot(gs[0, 0])  # Use the first column (taking 3/4 of space)

y_name = 'qx_norm'
res_list = L_res_list

for i, L in enumerate(L_range):
    res = res_list[i]
    tabl = res['tabl']
    
    tabl['x'] = ((0.5 + tabl.index) / (tabl.index[-1] + 1)) * hw_local.man.L

    ax1.plot(round(tabl['x'] * 1000, 1), tabl[y_name],label = f"$L_{{riser}} = {round(res['par']['L_riser']*1000,1)} \: mm$",
                color=colors[i],
                linestyle = linestyles[i],
                linewidth = 2.
    )

ax1.set_xlim(0,1.04*1000)
ax1.set_xlabel('Position along the manifold [mm]')
ax1.set_ylabel('Normalized flow rate [-]')
ax1.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

# Adding the legend below the first plot
fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=2, frameon=False)

# Second plot: 1/4 of the width (scatter plot from the previous example)
ax2 = fig.add_subplot(gs[0, 1])  # Use the second column (taking 1/4 of space)

test_range = L_range

ax2.scatter(test_range, [res['PL'] for res in res_list], color='grey')

ax2.set_xlabel('$R_D$ [-]')
ax2.set_ylabel('PL [Pa]')
ax2.grid()

# Adjust the layout to ensure better spacing between plots
plt.subplots_adjust(wspace=0.3)  # Increase horizontal space between plots

plt.show()

In [208]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_PL\PL-2.4-parametric-length-riser.png', dpi=600)

### Parametric E

In [194]:
path = r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Inputs"
file_name = "\V4.5_wo_interMPE.xlsx"
hw, par, cond = fe.initialize(path, file_name)

hw.change_N_total(16)

hw.riser.shape = 'tubular'
hw.change_riser_diameter(0.004)

In [ ]:
test_range = [10,20,100,150,200,250]

E_res_list = []

for i, test_value in enumerate(test_range):

    hw_local = copy.deepcopy(hw)

    hw_local.change_manifold_length_only(test_value * hw_local.man.D)

    hw_local.compute_metrics()
    par = hw_local.make_dict()

    print('E = ', hw_local.E)
    print('inter_riser =', hw_local.inter_riser)
    print('L_man ', hw_local.man.L)

    tabl, res, df_PL, residuals = modf.PL_fsolve(par, cond) 

    tabl['qx_norm'] = tabl['qx'] / (tabl['qx'].sum() / hw_local.N)

    E_res_list.append({'E' : test_value,
                       'par' : copy.deepcopy(par),
                       'tabl' : copy.deepcopy(tabl),
                       'PL' : res, 
                       'df_PL' : copy.deepcopy(df_PL),
                       'residuals' : copy.deepcopy(residuals),
                       }
                       )
    

In [ ]:
# Main plot: plotting L_f0_range on x-axis
fig, ax1 = plt.subplots(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))

y_name = 'qx_norm'

color_intensity = [70,100,150]

res_list = E_res_list

for i, E in enumerate(test_range):
    res = res_list[i]
    tabl = res['tabl']

    # tabl['x'] = ((0.5+tabl.index)/(tabl.index[-1]+1)) * hw_local.man.L

    ax1.plot(tabl.index, tabl[y_name], label = f"$E = {E}, L_{{man}} = {round(res['par']['L_man']*1000,1)} \: mm$",
            #  color = colors[i],
             linestyle = linestyles[i],
             linewidth = 2.
            )

ax1.set_xlabel('Position along the manifold [mm]')
ax1.set_ylabel('Normalized flow rate [-]')

# ax1.set_xlim(0,714.75)

ax1.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=2, frameon=False,)

